In [1]:
import os
import pandas as pd
import tensorflow as ts
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
from keras import backend as K
import pickle

In [2]:
os.chdir("../")
os.chdir("FinalDataset")

In [3]:
data = pd.read_csv("feature.csv")

In [4]:
data.head()

,Unnamed: 0,File,bodyLength,bscr,dse,dsr,entropy,hasHttp,hasHttps,has_ip,...,numParams,numTitles,num_%20,num_@,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
0,0,spam_dataset,31,0.064516,0,0,-4.708307,True,False,0,...,0,0,0,0,0.000000,0,2,0.000000,False,81
1,1,spam_dataset,64790,0.306050,0,0,-4.576882,True,False,0,...,0,19,0,0,0.923800,59853,19829,3.018458,False,58
2,2,spam_dataset,757,0.195509,0,0,-4.205536,True,False,0,...,0,1,0,0,0.000000,0,148,0.000000,False,46
3,3,spam_dataset,266310,0.230915,0,0,-4.550931,True,False,0,...,0,648,0,0,0.312647,83261,61495,1.353947,False,52
4,4,spam_dataset,40,0.200000,0,0,-4.689336,True,False,0,...,2,1,0,0,0.000000,0,8,0.000000,False,83


In [5]:
set(data['File'])

{'Benign_list_big_final',
 'Defacement',
 'Malware_dataset',
 'phishing_dataset',
 'spam_dataset'}

In [6]:
data.replace("Benign_list_big_final","Benign",inplace=True)
data.replace("Malware_dataset","Malware",inplace=True)
data.replace("phishing_dataset","Phishing",inplace=True)
data.replace("spam_dataset","Spam",inplace=True)

In [7]:
data.drop(columns='Unnamed: 0',inplace=True)
data.replace(True,1,inplace = True)
data.replace(False,0,inplace = True)
y = data["File"]

In [8]:
data = data.drop(columns = "File")

In [9]:
encoder = LabelEncoder()
encoder.fit(y)
Y = encoder.transform(y)

In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(data)
X = pd.DataFrame(X)

In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

input_dim = len(data.columns)
model = Sequential()
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [13]:
model.fit(X_train,np_utils.to_categorical(y_train),epochs = 50,validation_split=0.3, batch_size = 128)

Epoch 1/50
224/224 [==============================] - 1s 4ms/step - loss: 1.2592 - accuracy: 0.4481 - val_loss: 1.0809 - val_accuracy: 0.5421
Epoch 2/50
224/224 [==============================] - 1s 3ms/step - loss: 0.9683 - accuracy: 0.6110 - val_loss: 0.9297 - val_accuracy: 0.6162
Epoch 3/50
224/224 [==============================] - 1s 3ms/step - loss: 0.8358 - accuracy: 0.6728 - val_loss: 0.7926 - val_accuracy: 0.6917
Epoch 4/50
224/224 [==============================] - 1s 4ms/step - loss: 0.7545 - accuracy: 0.7004 - val_loss: 0.7121 - val_accuracy: 0.7214
Epoch 5/50
224/224 [==============================] - 1s 3ms/step - loss: 0.7032 - accuracy: 0.7188 - val_loss: 0.7806 - val_accuracy: 0.7011
Epoch 6/50
224/224 [==============================] - 1s 3ms/step - loss: 0.6738 - accuracy: 0.7381 - val_loss: 0.6602 - val_accuracy: 0.7372
Epoch 7/50
224/224 [==============================] - 1s 3ms/step - loss: 0.6469 - accuracy: 0.7468 - val_loss: 0.6695 - val_accuracy: 0.7249
Epoch 

In [14]:
y_pred = model.predict(X_test)

In [15]:
predicted = np.argmax(y_pred, axis=1)

In [16]:
from sklearn.metrics import classification_report
print(accuracy_score(y_test,predicted))

0.8437614611604196


In [17]:
target_names = ['Benign','Defacement','Malware','Phishing','Spam']
print(classification_report(y_test, predicted, target_names=target_names))

              precision    recall  f1-score   support

      Benign       0.91      0.96      0.93      2735
  Defacement       0.79      0.73      0.76      2499
     Malware       0.78      0.92      0.85      2834
    Phishing       0.84      0.71      0.77      2477
        Spam       0.89      0.86      0.88      3088

    accuracy                           0.84     13633
   macro avg       0.84      0.84      0.84     13633
weighted avg       0.84      0.84      0.84     13633



In [18]:
os.chdir("../")
os.chdir("models")
model.save("Model_v2.h5")
np.save('lblenc_v1.npy', encoder.classes_)
scalerfile = 'scaler_v1.sav'
pickle.dump(scaler, open(scalerfile, 'wb'))

In [19]:
import pathlib

In [20]:

model = ts.keras.models.load_model('Model_v2.h5')
converter = ts.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("tflite_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\candra\AppData\Local\Temp\tmpfs4bthpp\assets


200812

In [21]:
model = ts.keras.models.load_model('Model_v2.h5')
converter = ts.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [ts.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\candra\AppData\Local\Temp\tmpr3j9f3lr\assets


INFO:tensorflow:Assets written to: C:\Users\candra\AppData\Local\Temp\tmpr3j9f3lr\assets


In [22]:
tflite_model_file = pathlib.Path('tflite_quant_model.tflite')
tflite_model_file.write_bytes(tflite_quant_model)

56032